# Assessment: Unit 6
--------------------

Complete the problems below in your copy of the Jupyter Notebook.

## Problem 6.1.

## Problem 6.2.

## Problem 6.3.

--------------
## Next Steps:

1. Advance to [Unit 7](../07-advanced-plotting/unit07-lesson.ipynb) when you're ready for the next step